In [173]:
import numpy as n
import openpyxl
from scipy.stats import binom

In [174]:
## Import PoF and Failure Time
def Update():
  # Model_List must be imported every time
  data = openpyxl.load_workbook("Model_List.xlsx")
  read_data = data.active
  Probability = []
  Fail_Time = []
  for row in range(1, read_data.max_row):
      for col in read_data.iter_cols(2,2):
        Probability.append(col[row].value)
  for row in range(1, read_data.max_row):
      for col in read_data.iter_cols(3,3):
        Fail_Time.append(col[row].value)
  return [Probability, Fail_Time]


In [175]:
## Tests each probability given a binomial distribution
def test(Probability):
  return binom.rvs(1,Probability)


In [176]:
## Tests the Electrical side of the model. This is checked before HVAC.
def Failure_Elec(PoF,Carry):
  # 1,2,3,4 are the generators, only 2 are needed
  if(str(PoF)=="1" or str(PoF)=="2" or str(PoF)=="3" or str(PoF)=="4"):
    # Checks if there are 3 gens that fail
    if (Carry[1] + Carry[2] + Carry[3] + Carry[4]) == 3:
      Carry[PoF] += 1
      return [0,Carry]
    # If less than 2 fail, increases the carry
    Carry[PoF] += 1
    return [1,Carry]
  # Checks the busses, only one needed, same process as gens
  elif(str(PoF)=="5" or str(PoF)=="6"):
    if (Carry[5] or Carry[6]) == 1:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  # Checks UPS, only one needed
  elif(str(PoF)=="7" or str(PoF)=="8"):
    if (Carry[7] or Carry[8]) == 1:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  # If none of the components are those listed above, there is no exception
  # therefor they fail
  else:
    return [0,Carry]

In [177]:
# Checks HVAC side, same as electrical]
def Failure_HVAC(PoF,Carry):
  # Check for Watertowers
  if(str(PoF)=="30" or str(PoF)=="32" or str(PoF)=="34"):
    Carry[PoF] += 1
    return [1,Carry]
  # Checks Pumps
  elif(str(PoF)=="31" or str(PoF)=="33" or str(PoF)=="35"):
    Carry[PoF] += 1
    # Checks if those pumps that are connected to water towers are down together
    if((Carry[30]==0 and Carry[31]==0) or (Carry[32]==0 and Carry[33]==0)
      or (Carry[34]==0 and Carry[35]==0)):
      return [1,Carry]
    return [0,Carry]
  # Checks Direct Chillers
  elif(str(PoF)=="40" or str(PoF)=="37" or str(PoF)=="38" or str(PoF)=="39"):
    if (Carry[40] + Carry[37] + Carry[38] + Carry[39]) == 4:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  # Checks Backup Chillers
  elif(str(PoF)=="41" or str(PoF)=="42"):
    if (Carry[41] or Carry[42]) == 1:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  # Checks CRAH's
  elif(str(PoF)=="48" or str(PoF)=="49"):
    if (Carry[48] or Carry[49]) == 1:
      Carry[PoF] += 1
      return [0,Carry]
    Carry[PoF] += 1
    return [1,Carry]
  else:
    return [0,Carry]

In [178]:
## Runs the Simulation
def Simulation():
  # Allows for the import of the list of PoF and Failtime
  pull = Update()
  Prob = pull[0]
  Fail_Time = pull[1]
  # Sets the hour
  hour = 0
  # Sets the list for each hour
  Year_Output = []
  # Creates an empty list with the amount of components to count failures
  Fail_List = [0]*len(Prob)
  while hour < (24*100):
    i = 0
    test_mult = 1
    elec_mult = 0
    Comp_Test=[]
    # Tests every component and adds them to the Comp_Test list
    while i < len(Prob):
      temp = test(Prob[i])
      Comp_Test.append(temp)
      test_mult *= temp
      # Does the comp for only electrical
      if i == 29:
        elec_mult = test_mult
      i+=1
    # If every component passes, passes the hour
    if test_mult == 1 :
      Year_Output.extend([1])
      hour+=1
    else:
      # Sees if positions need to be skipped
      if (Comp_Test[0] == 1):
        # If all electrical is good, start with HVAC
        if elec_mult == 1:
          i = 29
        # Else, start after gens since com power is good
        else:
          i = 5
      # If Com Power is down, start with Com Power
      else:
        i = 0
      Fail_Carry = [0]*len(Prob)
      # Tests Electrical slide, last lectrical comp is 29
      while i < 30:
        # Does comp Fail
        if Comp_Test[i] == 0:
          temp = Fail_List[i]
          Fail_List[i] = temp + 1
          con = Failure_Elec(i,Fail_Carry)
          # Gets info from Failure_Elec and sees if failed
          if (con[0] == 0 and i!=0):
            Year_Output.extend([0]*Fail_Time[i])
            hour += Fail_Time[i]
            i = 30
          # Keeps the carry and continues
          else:
            Fail_Carry = con[1]
        i += 1
        # Catches loop if all comps work, starting with 30
        if i == 30:
          while i < (len(Prob)):
            if Comp_Test[i] == 0:
              temp = Fail_List[i]
              Fail_List[i] = temp + 1
              con = Failure_HVAC(i,Fail_Carry)
              if (con[0] == 0):
                Year_Output.extend([0]*Fail_Time[i])
                hour += Fail_Time[i]
                i = 49
              else:
                Fail_Carry = con[1]
            i+=1
  # Returns the Total amount of hours passed in a year and the list of fails
  return [sum(Year_Output) , Fail_List]


In [190]:
from numpy.polynomial.polynomial import polyval3d
def run_multi(years):
  i = 0
  Output = 0
  List = []
  while i < years:
    temp = Simulation()
    print(temp)
    Output = Output + temp[0]
    List.append(temp[1])
    i+=1
  Fail_List = [0]*50
  for row in range(years):
    for col in range(len(Fail_List)):
      temp = Fail_List[col]
      Fail_List[col] = temp + List[row][col]
  Output = (Output*100)/(years*24*365)
  print(Fail_List)
  print(Output)
  return Fail_List
 
  
  
 
 

In [191]:

run_multi(1)


[2394, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 13, 0, 17, 0, 0, 2, 0, 2, 0, 4, 1, 0, 0, 0, 0, 1, 1, 1]]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 13, 0, 17, 0, 0, 2, 0, 2, 0, 4, 1, 0, 0, 0, 0, 1, 1, 1]
27.328767123287673


[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 14,
 0,
 13,
 0,
 17,
 0,
 0,
 2,
 0,
 2,
 0,
 4,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1]

In [189]:
def temp_thing(lol):
 Final_Fail = []
 data = openpyxl.load_workbook("Model_List.xlsx")
 read_data = data.active
 i = -1
 for row in range(1, read_data.max_row):
     i +=1
     for col in read_data.iter_cols(1,1):
        temp_name = col[row].value
        Final_Fail.append([lol[i],temp_name])
        if(i == 49):
            return Final_Fail
            
        
    
        
        
 
output = [0,1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 15, 1, 10, 0, 0, 1, 4, 1, 3, 1, 0, 0, 0, 0, 1, 0, 1]
temp_thing(output)

[[0, 'XCEL Energy (Commercial Power)'],
 [1, 'Generator 1'],
 [0, 'Generator 2'],
 [0, 'Generator 3'],
 [0, 'Generator 4'],
 [0, 'Comercial Bus'],
 [0, 'Generator Bus'],
 [0, 'SS UPS 1'],
 [0, 'SS UPS 2'],
 [0, 'Swichgear Input 1'],
 [0, 'Swichgear Input 2'],
 [0, 'UPS 1B'],
 [0, 'UPS 1A'],
 [0, 'UPS 2B'],
 [0, 'UPS 2A'],
 [0, 'UPS 1 Switchboard'],
 [0, 'UPS 2 Switchboard'],
 [0, 'STS 2'],
 [0, 'Switchgear Output 2'],
 [0, 'Critical bus 2A'],
 [0, 'Critical bus 2B'],
 [0, 'Panel TB12A-TB3'],
 [0, 'PDU-B'],
 [0, 'PDU-A'],
 [0, 'STS'],
 [0, 'Panel'],
 [0, 'Panel DPA'],
 [0, 'Trans T4'],
 [0, 'Panel LPA 1'],
 [0, 'Panel LPA 2'],
 [0, 'Water Tower 1'],
 [14, 'Pump 1'],
 [0, 'Water Tower 2'],
 [15, 'Pump 2'],
 [1, 'Water Tower 3'],
 [10, 'Pump 3'],
 [0, 'Sump'],
 [0, 'Chiller 2'],
 [1, 'Chiller 1'],
 [4, 'Chiller 3'],
 [1, 'Chiller 6'],
 [3, 'Chiller 5'],
 [1, 'Chiller 4'],
 [0, 'Pump P3'],
 [0, 'Pump P4'],
 [0, 'CW Storage Tank'],
 [0, 'Pump P5'],
 [1, 'Pump '],
 [0, 'CRAH 1'],
 [1, 'CRAH 